In [1]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
import json
import plotly.graph_objs as go
import datetime

In [2]:
df2 = pd.read_feather('./data/topaz_data_w_bsp_new6.fth')

In [3]:
date_start = datetime.datetime(2020, 1, 1).date()
date_fin = datetime.datetime(2023, 1, 1).date()
df2 = df2.query('date >= @date_start & date <= @date_fin')

In [4]:
df2 = df2.query('state == "NZ"')

In [5]:
df2.to_csv('topaz_data_for_r_v7.csv', index=False)

Index(['meetingDate', 'state', 'track', 'distance', 'raceId', 'raceTypeCode',
       'raceNumber', 'boxNumber', 'rugNumber', 'runId',
       ...
       'prev_race_track', 'prev_race_state', 'next_race', 'prev_BSP_price',
       'prev_start_price', 'stats_topaz', 'dogid', 'raceid', 'stats_cols',
       'track_hash'],
      dtype='object', length=155)

In [67]:
df4 = pd.read_feather('./data/topaz_data_w_bsp_new4.fth')

In [68]:
df2 = df2.query('state == "NZ"')
df4 = df4.query('state == "NZ"')

In [69]:
date_start = datetime.datetime(2020, 1, 1).date()
date_fin = datetime.datetime(2023, 1, 1).date()
df2 = df2.query('date >= @date_start & date <= @date_fin')
df4 = df4.query('date >= @date_start & date <= @date_fin')


In [70]:
df2.columns

Index(['meetingDate', 'state', 'track', 'distance', 'raceId', 'raceTypeCode',
       'raceNumber', 'boxNumber', 'rugNumber', 'runId',
       ...
       'prev_race', 'prev_race_date', 'prev_race_track', 'prev_race_state',
       'next_race', 'stats_topaz', 'dogid', 'raceid', 'stats_cols',
       'track_hash'],
      dtype='object', length=144)

In [71]:
feature_cols = ['dogAgeScaled',
 'boxNumber',
 'weightInKgScaled',
 'hasEntryBoxNumberPlus1',
 'hasEntryBoxNumberMinus1',
 'rolling_box_win_percentage',
 'dog_distance_mean_1',
 'dog_boxNumber_mean_1',
 'dog_runTimeNorm_mean_1',
 'dog_place_mean_1',
 'dog_resultMargin_mean_1',
 'dog_split_time_margin_mean_1',
 'dog_split_runTimeNorm_mean_1',
 'dog_time_1_mean_1',
 'dog_run_home_TimeNorm_mean_1',
 'dog_finishingPlaceMovement_mean_1',
 'dog_averageSpeed_mean_1',
 'dog_win_mean_1',
 'trainer_distance_mean_365D',
 'trainer_boxNumber_mean_365D',
 'trainer_runTimeNorm_mean_365D',
 'trainer_place_mean_365D',
 'trainer_resultMargin_mean_365D',
 'trainer_split_time_margin_mean_365D',
 'trainer_split_runTimeNorm_mean_365D',
 'trainer_time_1_mean_365D',
 'trainer_run_home_TimeNorm_mean_365D',
 'trainer_finishingPlaceMovement_mean_365D',
 'trainer_averageSpeed_mean_365D',
 'trainer_win_mean_365D',
 'dam_distance_mean_365D',
 'dam_boxNumber_mean_365D',
 'dam_runTimeNorm_mean_365D',
 'dam_place_mean_365D',
 'dam_resultMargin_mean_365D',
 'dam_split_time_margin_mean_365D',
 'dam_split_runTimeNorm_mean_365D',
 'dam_time_1_mean_365D',
 'dam_run_home_TimeNorm_mean_365D',
 'dam_finishingPlaceMovement_mean_365D',
 'dam_averageSpeed_mean_365D',
 'dam_win_mean_365D',
 'sire_distance_mean_365D',
 'sire_boxNumber_mean_365D',
 'sire_runTimeNorm_mean_365D',
 'sire_place_mean_365D',
 'sire_resultMargin_mean_365D',
 'sire_split_time_margin_mean_365D',
 'sire_split_runTimeNorm_mean_365D',
 'sire_time_1_mean_365D',
 'sire_run_home_TimeNorm_mean_365D',
 'sire_finishingPlaceMovement_mean_365D',
 'sire_averageSpeed_mean_365D',
 'sire_win_mean_365D']

In [72]:
for c in feature_cols:
    zero_perc = df4[df4[c]==0].shape[0] / df4.shape[0] 
    print(f"Percentage of zero values in {c}: {zero_perc}")

Percentage of zero values in dogAgeScaled: 0.14418252143006483
Percentage of zero values in boxNumber: 0.0
Percentage of zero values in weightInKgScaled: 0.13775350198620112
Percentage of zero values in hasEntryBoxNumberPlus1: 0.873222872674054
Percentage of zero values in hasEntryBoxNumberMinus1: 1.0
Percentage of zero values in rolling_box_win_percentage: 0.00594118057007457
Percentage of zero values in dog_distance_mean_1: 0.01613352846888285
Percentage of zero values in dog_boxNumber_mean_1: 0.01613352846888285
Percentage of zero values in dog_runTimeNorm_mean_1: 0.017623179315631753
Percentage of zero values in dog_place_mean_1: 0.01613352846888285
Percentage of zero values in dog_resultMargin_mean_1: 0.016569098891908843
Percentage of zero values in dog_split_time_margin_mean_1: 0.9988588054916719
Percentage of zero values in dog_split_runTimeNorm_mean_1: 0.8778573419750505
Percentage of zero values in dog_time_1_mean_1: 0.8748780402815527
Percentage of zero values in dog_run_hom

In [9]:
for c in feature_cols:
    if 'Id' in c:
        print(c)
    elif c not in df4.columns:
        print(f"column: {c}, in df2, {c in df2.columns}")
    else:
        # if type df2[c]
        try:
            print(f"column: {c}, in df4, {c in df4.columns}, mean df2 = {df2[c].mean()}, mean df4 = {df4[c].mean()}, {df2[c].mean()- df4[c].mean()}")   
        except Exception as e:
            pass

column: dogAgeScaled, in df4, True, mean df2 = 0.443152085365721, mean df4 = 0.44302941760023573, 0.0001226677654852515
column: boxNumber, in df4, True, mean df2 = 4.49080044757697, mean df4 = 4.491661139012603, -0.0008606914356326101
column: weightInKgScaled, in df4, True, mean df2 = 0.49216475386501546, mean df4 = 0.4918686465796142, 0.0002961072854012481
column: hasEntryBoxNumberPlus1, in df4, True, mean df2 = 0.1254138356654247, mean df4 = 0.12485786032407846, 0.0005559753413462293
column: hasEntryBoxNumberMinus1, in df4, True, mean df2 = 0.0, mean df4 = 0.0, 0.0
column: rolling_box_win_percentage, in df4, True, mean df2 = 0.1294481756415974, mean df4 = 0.12867077655932826, 0.0007773990822691479
column: dog_distance_mean_1, in df4, True, mean df2 = 379.84491688680225, mean df4 = 379.6427674594902, 0.2021494273120652
column: dog_boxNumber_mean_1, in df4, True, mean df2 = 4.422095075499775, mean df4 = 4.424109258030891, -0.002014182531116404
column: dog_runTimeNorm_mean_1, in df4, Tr

In [ ]:
df = pd.read_feather('model_all_price/NZ-dark-cosmos-774 - val_all_price_df.fth')
df.to_csv('NZ-dark-cosmos-774 - val_all_price_df.csv')

In [ ]:
df = pd.read_feather('./data/topaz_data_w_bsp_new_w_price.fth')

In [ ]:
for i in df.columns:
    print(i)

In [ ]:
prev_stat = df.stats_topaz.apply(lambda x: x[0])

In [ ]:
prev_stat

In [ ]:
df.stats_topaz.iloc[1_000_000].round(5).tolist()

In [ ]:
df = df.query('state == "VIC"')

In [ ]:
df = df[df['track'].isin(["Bendigo", "Geelong", "Traralgon"])]
df.prev_race_track.value_counts()

In [ ]:
prev_race_not_vic = df.query('prev_race_state != "VIC"')

In [ ]:
prev_race_not_vic.prev_race_track.value_counts()

In [ ]:
model_name  = 'VIC-youthful-fog-761'
epoch = '600'
all_price_df = pd.read_feather(f'model_all_price/{epoch}{model_name} - val_all_price_df.fth')

In [34]:
all_price_df.favorite_correct.value_counts()

favorite_correct
0.0    45308
1.0     2652
Name: count, dtype: int64

In [32]:
wrong = all_price_df.query('favorite_correct == 1 and correct == 0')


In [33]:
wrong_race_ids = wrong['raceID'].tolist()
wrong_race_ids

['857095230',
 '857095228',
 '857095226',
 '861001091',
 '861001078',
 '861001087',
 '860994588',
 '861001206',
 '860650646',
 '860650644',
 '860994767',
 '860994805',
 '861001396',
 '860650858',
 '861001494',
 '861001486',
 '863837782',
 '860994956',
 '864267639',
 '864267629',
 '864267633',
 '865722017',
 '865722013',
 '865622378',
 '865622373',
 '865722018',
 '865722065',
 '865722069',
 '865722137',
 '865722134',
 '865722196',
 '865722195',
 '865622474',
 '865722193',
 '865722188',
 '865622579',
 '865622578',
 '865622577',
 '865598076',
 '865598071',
 '865598121',
 '865598123',
 '865598153',
 '865598146',
 '865598143',
 '865598136',
 '865598131',
 '868873784',
 '869415419',
 '869915867',
 '869915888',
 '869915878',
 '869915882',
 '869915873',
 '869915957',
 '869915995',
 '869916001',
 '869953521',
 '869953517',
 '870434536',
 '870731115',
 '870434542',
 '870731105',
 '870731114',
 '870678618',
 '870678613',
 '870731280',
 '870678726',
 '870678724',
 '870678812',
 '870678814',
 '8706

In [35]:
wrong_df = all_price_df[all_price_df['raceID'].isin(wrong_race_ids)]

In [ ]:
single_dog = all_price_df.query('dogID == "263426803"')
# single_dog.to_csv('single_dog.csv')

In [ ]:
single_dog 

In [ ]:
df = pd.read_feather('./data/topaz_data_w_bsp_new4.fth')

In [ ]:
df.query('dogId == 782101719').to_csv('single_dog_nz.csv')

In [ ]:
df.query('dogId == 740359894')

In [ ]:
df.BSP.isna().sum()

In [ ]:
df_w_prices = df.query('BSP.notna()')

In [ ]:
df_w_prices.BSP.isna().sum()

In [ ]:
runs_df = df_w_prices
runs_df['year-month'] = pd.to_datetime(runs_df['meetingDate']).dt.to_period('M')
runs_df['year'] = pd.to_datetime(runs_df['meetingDate']).dt.to_period('Y')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming runs_df is your DataFrame
# Make sure 'year-month' is in datetime format
# runs_df['year-month'] = pd.to_datetime(runs_df['year-month'])

# Calculate average win price and BSP for place == 1
# avg_win_price = runs_df[runs_df['place'] == 1].groupby('year-month')['win price'].mean()

avg_BSP = runs_df[runs_df['place'] == 1][runs_df['state'] == 'NSW'].groupby('year-month')['BSP'].median()

# Create a new DataFrame for the averages
avg_df = pd.DataFrame({ 'Average BSP': avg_BSP})

# Plot the averages
avg_df.plot(kind='line', marker='o')
plt.title('Average Win Price and BSP Over Time')
plt.xlabel('Year-Month')
plt.ylabel('Average Price')
plt.show()

In [ ]:
df.query('selectionI == 914237037').to_csv('blahblach.csv')

In [ ]:
df.query('raceId == 914237037')

In [ ]:
df.columns

In [ ]:
df.dogId

In [36]:
wrong_df['raceID'] = wrong_df['raceID'].astype(int)
wrong_df['dogID'] = wrong_df['dogID'].astype(int)

C:\Users\Nick\AppData\Local\Temp\ipykernel_20740\2040291442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrong_df['raceID'] = wrong_df['raceID'].astype(int)
C:\Users\Nick\AppData\Local\Temp\ipykernel_20740\2040291442.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrong_df['dogID'] = wrong_df['dogID'].astype(int)


In [38]:
wrong_race_merge_stats = wrong_df.merge(df4, left_on=['raceID', 'dogID'], right_on=['raceId', 'dogId'])

In [39]:
wrong_race_merge_stats.to_csv('wrong_race_merge_stats.csv')

In [ ]:
wrong_race_merge_stats.track_x.value_counts()

In [ ]:
wrong.win_price.count()

In [ ]:
all_price_df.query('favorite_correct == 0 and correct == 1').win_price.count()


In [ ]:
all_price_df.to_excel(f'model_all_price/{epoch}{model_name} - all_price_df.xlsx')

In [ ]:
    # model_all_price/RL40rich-brook-43 - val_all_price_df.fth model_all_price/RL40rich-brook-43 - all_price_df.fth

In [ ]:
all_price_df.columns

In [ ]:
# all_price_df[['betfair_log_loss', 'log_loss', 'label_loss']] = all_price_df[['betfair_log_loss', 'log_loss', 'label_loss']].expanding().mean()
bydate = all_price_df.groupby(['date','raceID'])[['betfair_log_loss', 'log_loss', 'label_loss']].sum().reset_index().groupby('date').mean(numeric_only=True).expanding().mean().reset_index()
# bydate = bydate.iloc[100_000:]

In [ ]:
bydate

In [ ]:
import plotly.graph_objects as go

# Create a trace for ROI
trace_roi = go.Scatter(
    x = bydate['date'],
    y = bydate['betfair_log_loss'],
    mode = 'lines',
    name = 'betfair_log_loss',
    yaxis='y1'  # This associates this trace with the first y-axis
)

# Create a second trace for the second y-axis
trace_second = go.Scatter(
    x = bydate['date'],
    y = bydate['log_loss'],  # Replace with your second data
    mode = 'lines',
    name = 'log_loss',
    yaxis='y1'  # This associates this trace with the second y-axis
)

data = [trace_roi, trace_second]

# Define layout
layout = go.Layout(
    title = 'Interactive Plot',
    xaxis = dict(title = 'Round Price'),
    yaxis = dict(title = 'Amount'),
    yaxis2=dict(
        title='Second Amount',  # Title for the second y-axis
        overlaying='y',  # This line and the line below ensure that the second y-axis is displayed on the right
        side='right'
    )
)

# Define the figure
fig = go.Figure(data=data, layout=layout)

# Show the figure
fig.show()

In [4]:
model_name  = 'NZ-fancy-dawn-14'
epoch = ''
all_price_df = pd.read_feather(f'model_all_price/{epoch}{model_name} - all_price_df.fth')
# all_price_df
all_price_df2 = pd.read_feather(f'model_all_price/{epoch}{model_name} - val_all_price_df.fth')
all_price_df = pd.concat([all_price_df, all_price_df2])

In [5]:
all_price_df.to_csv(f'model_all_price/{epoch}{model_name} - all_price_df.csv')

In [51]:
all_price_df.columns

Index(['index', 'raw_margins', 'correct', 'simple', 'win_price', 'relu',
       'pred_order', 'output_logits', 'bet_amount_model', 'output_price',
       'pred_prob', 'pred_prob2', 'prices', 'imp_prob', 'betfair_log_loss',
       'log_loss', 'label_loss', 'pred_price', 'pred_price2', 'classes',
       'track', 'onehot_win', 'dogID', 'dog_name', 'dog_box', 'raceID', 'date',
       'entropy', 'mutual_info', 'race_num', 'loss', 'loss_bfsp',
       'favorite_correct', 'one_hot_win', 'bet_amount', 'profit',
       'seq_kelly_ratio', 'bet_kelly_seq', 'profit_kelly_sq', 'colour',
       'profit < 30', 'outlay < 30', 'bet_amount2', 'profit2',
       'bet_amount_kelly', 'profit_kelly', 'profit < 30 2', 'outlay < 30 2',
       'bet_relu', 'profit_relu', 'bet_relu<30', 'profit_relu<30', 'kl_div',
       'kl_div_bfsp', 'win < 30', 'win', 'bet_count', 'bet_count < 30',
       'profit_model', 'round_price'],
      dtype='object')

In [49]:
all_price_df.groupby('pred_order')['one_hot_win'].value_counts()

pred_order  one_hot_win
0           0              4061
            1              2488
1           0              5168
            1              1382
2           0              5626
            1               917
3           0              5863
            1               661
4           0              5953
            1               483
5           0              5799
            1               348
6           0              5168
            1               188
7           0              3773
            1                82
Name: count, dtype: int64

In [21]:
all_price_df.value_counts('pred_order')

pred_order
2    6163
0    6089
4    6058
1    6056
3    6012
5    5998
6    5885
7    5699
Name: count, dtype: int64

In [26]:
# Assuming df is your DataFrame
filtered_df = all_price_df.groupby('raceID').filter(lambda x: all(x['pred_order'] != 0))

unique_raceIds = filtered_df['raceID'].unique()

In [29]:
simple_df = all_price_df.query('raceID == "857095230"')

In [30]:
simple_df.pred_prob.sum()

0.984665797057799

In [ ]:
bytrack = all_price_df.groupby(['date','raceID','track'])[['betfair_log_loss', 'log_loss', 'label_loss']].sum().reset_index().groupby('track').mean(numeric_only=True)

In [ ]:
all_price_df.columns

In [ ]:
all_price_df.groupby('raceID')[['pred_prob']].sum().mean()

In [ ]:
bytrack['delta'] = bytrack['betfair_log_loss'] - bytrack['log_loss']
bytrack.plot.scatter(x='betfair_log_loss', y='delta')

In [ ]:
all_price_df.columns

In [ ]:
all_price_df.simple.sum()

In [ ]:
all_price_df.one_hot_win.sum()

In [ ]:
all_price_df['correct_over_fav'] = all_price_df['correct'] - all_price_df['favorite_correct']

In [ ]:
all_price_df['correct_over_fav'].sum()

In [ ]:
all_price_df.query('correct == 1')['prices'].mean()

In [ ]:
all_price_df.query('favorite_correct == 1')['prices'].mean()

In [ ]:
all_price_df

In [ ]:
all_price_df['los_loss_2'] = (-(all_price_df['label_loss']*np.log(all_price_df['pred_prob2'])+(1-all_price_df['label_loss'])*np.log(1-all_price_df['pred_prob2'])))

In [ ]:
all_price_df[['log_loss', 'los_loss_2']]

In [ ]:
all_price_df['los_loss_2'].mean()

In [ ]:
all_price_df['log_loss'].mean()

In [ ]:
all_price_df.groupby('raceID')[['los_loss_2']].sum().mean()

In [ ]:
all_price_df.groupby('raceID')[['log_loss']].sum().mean()

In [ ]:
import torch.nn.functional as F
import torch
def pd_softmax(x):
    x = torch.tensor(list(x))
    x = F.softmax(x/0.8, dim=0)
    return x.numpy()

In [ ]:
all_price_df.columns

In [ ]:
new_softmax = all_price_df.groupby('raceID',sort=False).apply(lambda x: pd_softmax(x['output_logits']))
all_price_df['new_pred'] = np.concatenate(new_softmax)

In [ ]:
all_price_df[['log_loss', 'los_loss_2', 'pred_prob', 'new_pred']]

In [ ]:
all_price_df['log_loss_new'] = (-(all_price_df['label_loss']*np.log(all_price_df['new_pred'])+(1-all_price_df['label_loss'])*np.log(1-all_price_df['new_pred'])))

In [ ]:
all_price_df[['log_loss', 'los_loss_2','log_loss_new', 'pred_prob', 'new_pred']].mean()

In [ ]:
all_price_df.groupby('raceID')[['log_loss', 'los_loss_2','log_loss_new','betfair_log_loss']].sum().mean()

In [ ]:
print(f"{all_price_df['profit_model < 30'].sum()=}")
print(f"{all_price_df2['profit_model < 30'].sum()=}")

In [ ]:
all_price_df['dt'] = pd.to_datetime(all_price_df['date']).dt.date
all_price_df = all_price_df.sort_values('dt',ascending=True)

In [ ]:
all_price_df = all_price_df2

In [ ]:
all_price_df['lay_profit_model'] = all_price_df['lay_amount_model']-all_price_df['onehot_win']*all_price_df['lay_amount_model']*all_price_df['prices']

In [ ]:
all_price_df['lay_profit_model'].sum()

In [ ]:
bet_all_df = all_price_df[all_price_df['bet_amount_model'] >= 0.6]

In [ ]:
bet_all_df['bet_under_30'] = np.where(bet_all_df['prices'] < 30,1,0)

In [ ]:
bet_all_df['bet_under_30'].mean()

In [ ]:
all_price_df.to_excel(f'model_all_price/{epoch}{model_name} - val-all_price_df.xlsx')

In [ ]:
model_name  = 'sage-blaze-232'
epoch = 200
all_price_df = pd.read_feather(f'model_all_price/{epoch}{model_name} - all_price_df.fth')
all_price_df
all_price_df2 = pd.read_feather(f'model_all_price/{epoch}{model_name} - val_all_price_df.fth')
all_price_df = pd.concat([all_price_df, all_price_df2])

In [ ]:
all_price_df.date.min()


In [ ]:
all_price_df.columns

In [ ]:
all_price_df.raceID.nunique()

In [ ]:
all_price_df.race_num.astype(int).value_counts(sort=False)

In [ ]:
all_price_df.to_parquet(f'model_all_price/{model_name} - all_price_df.pqt')

In [ ]:
all_price_df.shape

In [ ]:
all_price_df.to_excel(f'model_all_price/{model_name} - all_price_df.xlsx')

In [ ]:
model_name = 'faithful-sunset-63'
all_price_df_nz = pd.read_parquet(f'model_all_price/{model_name} - all_price_df.pqt')
all_price_df = all_price_df_nz

In [ ]:
import matplotlib.pyplot as plt

# Filter the dataframe where 'bet_amount_model' > 0.5
filtered_df = all_price_df[all_price_df['bet_amount_model'] > 0.8]
filtered_df = filtered_df[filtered_df['round_price'] < 100]
weights = np.ones_like(filtered_df['round_price']) / len(filtered_df['round_price'])

# Plot histogram with weights
plt.hist(filtered_df['round_price'], weights=weights, bins=50, edgecolor='black')

plt.xlabel('Round Price')
plt.ylabel('Percentage of bet_amount_model > 0.5')
plt.title('Histogram of percentages of bet_amount_model > 0.5 against round_price')
plt.show()

In [ ]:
filtered_df['round_price'].mean()

In [ ]:
filtered_df

In [ ]:
filtered_df = all_price_df[all_price_df['lay_amount_model'] > 0.8]
filtered_df = filtered_df[filtered_df['round_price'] < 30].reset_index(drop=True).copy()
average_price_per_day = filtered_df.groupby('date')['round_price'].mean()

# Plot histogram
plt.hist(average_price_per_day, bins=30, edgecolor='black')

plt.xlabel('Average Price')
plt.ylabel('Frequency')
plt.title('Histogram of average prices per day')
plt.show()

In [ ]:
all_price_df = pd.concat([all_price_df, all_price_df_nz])

In [ ]:
all_price_df = pd.read_excel(f'model_all_price/{model_name} - all_price_df.xlsx')

In [ ]:
price_df  = all_price_df.groupby('round_price')[['lay_amount_model', 'lay_profit_model','bet_amount_model', 'profit_model']].sum().cumsum().reset_index()


In [ ]:
import plotly.graph_objects as go
price_df['roi'] = price_df['profit_model'] / price_df['bet_amount_model']
price_df = price_df.query('round_price < 100.0')
# Create a trace for ROI
trace_roi = go.Scatter(
    x = price_df['round_price'],
    y = price_df['profit_model'],
    mode = 'lines',
    name = 'Profit Model'
)

data = [trace_roi]

# Define layout
layout = go.Layout(
    title = 'Interactive Plot',
    xaxis = dict(title = 'Round Price'),
    yaxis = dict(title = 'Amount'),
)

# Define the figure
fig = go.Figure(data=data, layout=layout)

# Show the figure
fig.show()


In [ ]:
all_price_df['dt'] = pd.to_datetime(all_price_df['date']).dt.date
all_price_df = all_price_df.sort_values('dt',ascending=True)

In [ ]:
model_name  = 'fiery-haze-173'
all_price_df2 = pd.read_feather(f'model_all_price/{epoch}{model_name} - val_all_price_df.fth')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import glob
import re
model_name  = 'fiery-haze-173'
files = glob.glob(f'model_all_price/*{model_name}* - val_all_price_df.fth')

# Create a figure and a single axis
fig, ax1 = plt.subplots(figsize=(10, 6))

colors = sns.color_palette('husl', len(files))
for i,file in enumerate(files):
    all_price_df = pd.read_feather(file)
    all_price_df['dt'] = pd.to_datetime(all_price_df['date']).dt.date

    date = all_price_df.query('round_price <= 24.0').copy()

    date['bet_amount_model'] = np.where(date['bet_amount_model'] > 0.1, date['bet_amount_model'], 0)*10*2
    date['profit_model'] = np.where(date['bet_amount_model'] > 0.1, date['profit_model'], 0)*10*2

    date = date.groupby('dt',sort=False)[['bet_amount_model', 'profit_model','round_price']].sum().cumsum().reset_index()
    date['roi'] = date['profit_model'] / date['bet_amount_model']
    epoch_number = re.search(r'\d+', file).group()
    # Plot the 'Profit Model' on the first axis
    color = colors[i]
    ax1.plot(date['dt'], date['profit_model'], label=f'Profit Model {epoch_number}', color=color)
    ax1.set_xlabel('Round Price')
    ax1.set_ylabel('Amount', color=color)

    # Create a second axis that shares the same x-axis
    # ax2 = ax1.twinx()

    # Plot the 'roi' on the second axis
    color = sns.color_palette()[1]
    # ax2.plot(date['dt'], date['roi'], label=f'roi {file}', color=color)
    # ax2.set_ylabel('roi', color=color)

# Show the legend
fig.legend(loc="upper left")

# Show the plot
plt.show()

In [ ]:
all_price_df.columns

In [ ]:
price_df  = all_price_df.query('round_price < 30').groupby('raceID')[['bet_amount_model', 'profit_model']].sum()

In [ ]:
price_df

In [ ]:
all_price_df.query('round_price < 30')['profit_model'].mean()

In [ ]:
all_price_df['roi'] = all_price_df['profit_model'] / all_price_df['bet_amount_model']

In [ ]:
price_df  = all_price_df.query('round_price < 30').groupby('raceID')[['bet_amount_model', 'profit_model']].sum()

In [ ]:
price_df

In [ ]:
price_df.mean()

In [ ]:
import scipy.stats as stats
model_name  = 'fiery-haze-173'
# file
# Concatenate all profit_model data from all files
# all_profits =  pd.read_parquet('model_all_price/fiery-haze-173 - all_price_df.pqt')['profit_model']
# all_price_df = price_df
all_price_df['roi'] = all_price_df['profit_model'] / all_price_df['bet_amount_model']
# all_price_df = all_price_df.query('bet_amount_model > 0.10 and prices < 30').copy()

all_profits = all_price_df['roi']
all_profits.dropna(inplace=True)

# Perform the t-test
t_stat, p_value = stats.ttest_1samp(all_profits, 0.15)

print(f'T-statistic: {t_stat}')
print(f'P-value: {p_value}')

In [ ]:
all_profits

In [ ]:

all_profits = all_price_df.groupby('raceID')[['bet_amount_model','profit_model']].sum()
all_profits['roi'] = all_profits['profit_model'] / all_profits['bet_amount_model']
all_profits = all_profits['roi']

In [ ]:
import numpy as np
all_profits 
std_dev = all_profits.std()
num = all_profits.count()
print(f'Standard deviation: {std_dev}')

Z = 1.96  # z-score for 95% confidence level
E = 0.1  # desired margin of error

# Calculate the sample size
n = (Z**2 * std_dev**2) / E**2
n = np.ceil(n)  # round up to the nearest whole number because you can't have a fraction of a sample

print(f'Sample size: {n}, num = {num}')

In [ ]:
win_df = all_price_df.query('round_price < 30.0').groupby('dt')['profit_model'].sum().reset_index()
win_df['win'] = np.where(win_df['profit_model'] > 0, 1, 0)
win_df['win'].value_counts()

In [ ]:
(Z**2 * std_dev**2)

In [ ]:
all_profits.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the seaborn color palette as the default matplotlib color palette
sns.set_palette('husl')

# Your existing code
date = all_price_df.query('round_price < 31.0').copy()

date['bet_amount_model'] = np.where(date['bet_amount_model'] > 0.1, date['bet_amount_model'], 0)*10*2
date['profit_model'] = np.where(date['bet_amount_model'] > 0.1, date['profit_model'], 0)*10*2

date = date.groupby('dt',sort=False)[['bet_amount_model', 'profit_model','round_price']].sum().cumsum().reset_index()
date['balance'] = date['profit_model']
date['adj_bet'] = np.where(date['balance']*0.05>20, date['balance']*0.05,20)
date['adj_profit'] = date['adj_bet']*date['profit_model']
date['roi'] = date['profit_model'] / date['bet_amount_model']
date['adj_roi'] = date['adj_profit'] / (date['adj_bet'].cumsum()*20)
# Create a figure and a single axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the 'Profit Model' on the first axis
color = sns.color_palette()[0]
ax1.plot(date['dt'], date['adj_profit'], label='Profit Model', color=color)
ax1.set_xlabel('Round Price')
ax1.set_ylabel('Amount', color=color)

color = sns.color_palette()[2]
ax1.plot(date['dt'], date['profit_model'], label='Profit Model', color=color)
ax1.set_xlabel('Round Price')
ax1.set_ylabel('Amount', color=color)


# Create a second axis that shares the same x-axis
ax2 = ax1.twinx()

# Plot the 'roi' on the second axis
color = sns.color_palette()[1]
ax2.plot(date['dt'], date['adj_roi'], label='roi', color=color)
ax2.set_ylabel('roi', color=color)

# Show the legend
fig.legend(loc="upper left")

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the seaborn color palette as the default matplotlib color palette
sns.set_palette('husl')

# Your existing code
date = all_price_df.query('round_price < 31.0').copy()
start_date = '2023-07-01'
start_date = pd.to_datetime(start_date).date()
date = date.query('dt >= @start_date').copy()
# date['bet_amount_model'] = date['lay_amount_model']# + date['bet_amount_model']
# date['profit_model'] = date['lay_profit_model']# + date['profit_model']
# date['bet_amount_model'] = np.where(date['bet_amount_model'] > 0.1, date['bet_amount_model'], 0)*1
# date['profit_model'] = np.where(date['bet_amount_model'] > 0.1, date['profit_model'], 0)*1
# date['profit_model'] = np.where(date['profit_model']>0,date['profit_model'],date['profit_model'])
date = date.groupby('dt',sort=False)[['bet_amount_model', 'profit_model','round_price']].sum().cumsum().reset_index()
date['roi'] = date['profit_model'] / date['bet_amount_model']

# Create a figure and a single axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Create a figure and a single axis
fig, ax1 = plt.subplots(figsize=(10, 6))
prices = [3,5,10,15,20,25,30,31,32,35,50,100,10000]
colors = sns.color_palette('husl', len(prices))
for i,price in enumerate(prices):
    print(f"{price=}")
    date = all_price_df.query('round_price < @price').copy()
    start_date = '2023-07-01'
    start_date = pd.to_datetime(start_date).date()
    date = date.query('dt >= @start_date').copy()
    date = date.groupby('dt',sort=False)[['bet_amount_model', 'profit_model','round_price']].sum().cumsum().reset_index()
    # Plot the 'Profit Model' on the first axis
    color = colors[i]
    ax1.plot(date['dt'], date['profit_model'], label=f'Profit Model {price}', color=color)
    ax1.set_xlabel('Round Price')
    ax1.set_ylabel('Amount', color=color)

    # Create a second axis that shares the same x-axis

    ax2 = ax1.twinx()

    # Plot the 'roi' on the second axis
    color = sns.color_palette()[1]
    # ax2.plot(date['dt'], date['roi'], label='roi', color=color)
    # ax2.set_ylabel('roi', color=color)

    # Show the legend
fig.legend(loc="upper left")

    # Show the plot
plt.show()

In [ ]:
#Testing difference in data between server and local
local = pd.read_feather('./data/gru_inputs_kitchen_sink_updated_recent.fth')
server = pd.read_feather('./data/testing new outs simple 2024-03-24_NZ.fth')

In [ ]:
local = local.rename({'stats':'stats_local','dog_id':'dogid'}, axis=1)
server = server.rename({'stats':'stats_server'}, axis=1)

In [ ]:
local.query('raceid=="1009424823"')

In [ ]:
local = local[['dogid','raceid','stats_local','prev_race_date']]
merged = server.merge(local, on=['dogid','raceid'], how='left')

In [ ]:
local.query('dogid == "760119322"')

In [ ]:
merged['stats_diff'] = merged.apply(lambda x: (x['stats_local'] - x['stats_server']).sum(), axis=1)

In [ ]:
dogid = "505983450"
stats_local = merged.query('dogid == @dogid')['stats_local'].to_list()[0]
stats_server = merged.query('dogid == @dogid')['stats_server'].to_list()[0]


In [ ]:
stats_local.round(2).tolist()

In [ ]:
delta = pd.DataFrame([stats_server.round(2)[6:],stats_local.round(2)[6:]],columns=eval(merged['stats_cols'][0]))

In [ ]:
local = local.query('state == "NZ"')

In [ ]:
local

In [ ]:
server['stats_cols'][0]

In [ ]:
df = df[[col for col in df.columns if '2' not in col and '<' not in col]]

In [ ]:
df['bet_kelly_seq'] = np.where(
    df['prices']<30,
    df['seq_kelly_ratio']*100,
    0
)


df['profit_kelly_sq'] = np.where(
    df['onehot_win'],
    df['bet_kelly_seq']  * (df['prices'] - 1) * 0.95,
    -1 * df['bet_kelly_seq']
)

In [ ]:
df_30 = df[df['prices']<30]

In [ ]:
df_30['profit_kelly_sq'].sum()/df_30['bet_kelly_seq'].sum()

In [ ]:
df_30['profit_kelly'].sum()/df_30['bet_amount_kelly'].sum()

In [ ]:
json_path = 'wandb_jsons/dancing-snake-80' 
files = os.listdir(json_path)
file_paths = [json_path + '/' + file for file in files]

for i,file in enumerate(file_paths):
    with open(file, 'r') as f:
        json_obj = json.load(f)

    # Create a DataFrame from the dictionary
    if i == 0:
        data = pd.DataFrame(data=json_obj["data"], columns=json_obj["columns"])
        data['step'] = i
    else:
        df = pd.DataFrame(data=json_obj["data"], columns=json_obj["columns"])
        df['step'] = i
        data = pd.concat([data, df])

In [ ]:
data

In [ ]:
df.filter

In [ ]:
df = data
df = data.query('onehot_win ==1')
# df = df.head(1000)

# Create a scatter plot for each step
# Create a scatter plot for each step
# Create a scatter plot for each step using Scattergl for WebGL
traces = [go.Scattergl(
    x=df['loss'][df['step'] == step].head(100),
    y=df['mutual_info'][df['step'] == step].head(100),
    mode='markers',
    visible=False,
    name=f'Step {step}'
) for step in df['step'].unique()]

# Make the first trace visible
traces[0]['visible'] = True

x_range = [df['loss'].min(), df['loss'].max()]
y_range = [df['mutual_info'].min(), df['mutual_info'].max()]

# Create a layout with a slider
layout = go.Layout(
    title='Loss vs Mutual Info',
    xaxis=dict(title='Loss'),
    yaxis=dict(title='Mutual Info'),
    sliders=[dict(
        active=0,
        currentvalue={"prefix": "Step: "},
        pad={"t": 50},
        steps=[dict(method='update',
                    args=[{'visible': [s == step for s in df['step'].unique()]}],
                    label=str(step)) for step in df['step'].unique()]
    )]
)

# Create a Figure
fig = go.Figure(data=traces, layout=layout)

# Display the figure
fig.show()